In [ ]:
rsync -avzhe ssh \
conchae@garnatxa.srv.cpd:/home/conchae/prediction_depolymerase_tropism/prophage_prediction/depolymerase_decipher/ficheros_28032023/train_nn/fine_tuning \
/media/concha-eloko/Linux/PPT_clean/in_vitro/fine_tuning

rsync -avzhe ssh \
conchae@garnatxa.srv.cpd:/home/conchae/prediction_depolymerase_tropism/prophage_prediction/depolymerase_decipher/ficheros_28032023/train_nn/fine_tuning/models_01 \
/media/concha-eloko/Linux/PPT_clean/in_vitro/fine_tuning

> Get the models at the right place : 

model     LR     weight decay
model     0.0001     0.000001
model_01  1e-6       0.001

In [1]:
import os 

path_ensemble = "/media/concha-eloko/Linux/PPT_clean/ficheros_28032023/ensemble_2411_finetuned"
path_base = "/media/concha-eloko/Linux/PPT_clean/ficheros_28032023/ensemble_2809"
path_finetuned = "/media/concha-eloko/Linux/PPT_clean/in_vitro/fine_tuning/fine_tuning/models_01"

# Move the desired epoch to the ensemble :
epoch = "9"
for model in os.listdir(path_finetuned) : 
    if model.split(".")[1] == epoch :
        os.system(f"cp {path_finetuned}/{model} {path_ensemble}")


In [2]:
# Move the rest :
finetuned_kltypes = [model.split(".")[0] for model in os.listdir(path_ensemble)]
for model in os.listdir(path_base) : 
    if model.split(".")[0] not in finetuned_kltypes : 
        os.system(f"cp {path_base}/{model} {path_ensemble}")

***

In [3]:
# Torch geometric modules
from torch_geometric.data import HeteroData, DataLoader
import torch_geometric.transforms as T
from torch_geometric.nn import to_hetero , HeteroConv , GATv2Conv
from torch_geometric.utils import negative_sampling
from torch_geometric.loader import LinkNeighborLoader

# Torch modules
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau

# SKlearn modules
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import LabelEncoder , label_binarize , OneHotEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score , matthews_corrcoef

# Ground modules
import os
import pandas as pd
import numpy as np
from tqdm import tqdm
from itertools import product
import random
from collections import Counter
import warnings
import logging
from multiprocessing.pool import ThreadPool

# TropiGAT modules
import TropiGAT_graph
import TropiGAT_models
import TropiGAT_functions

warnings.filterwarnings("ignore")

In [4]:
path_ensemble = "/media/concha-eloko/Linux/PPT_clean/ficheros_28032023/ensemble_2411_finetuned"

dico_models = TropiGAT_functions.make_ensemble_TropiGAT(path_ensemble)

> Others 

In [5]:
import pandas as pd 
import os 

path_project = "/media/concha-eloko/Linux/PPT_clean/in_vitro"

others_embeddings = pd.read_csv(f"{path_project}/Others_all.esm2.embedding.csv", sep = "," , header = None)
others_embeddings.set_index([0], inplace = True)

In [6]:
# Run the predictions Others :
other_predictions = {}
for index, dpo in enumerate(others_embeddings.index) :
    if dpo not in ["MN781108.1_prot_QGZ15323.1_262"] :
        try : 
            graph_dpo = TropiGAT_functions.make_query_graph([others_embeddings.loc[dpo].values])
            pred = TropiGAT_functions.run_prediction(graph_dpo,dico_models)
            other_predictions[dpo] = pred
        except Exception as e :
            print(e, dpo)

 ON146449.1_prot_UPW35150.1_13
 ON146449.1_prot_UPW35138.1_1
 ON146449.1_prot_UPW35150.1_13
 ON146449.1_prot_UPW35138.1_1


In [7]:
# format the results : 
others_pred_formated = TropiGAT_functions.format_predictions(other_predictions , sep = "_prot_")
TropiGAT_functions.clean_print(other_predictions)

{'AB716666.1_prot_BAP15736.1_24': {'KL110': 0.7704,
                                   'KL123': 0.9985,
                                   'KL125': 0.8789,
                                   'KL127': 0.5852,
                                   'KL14': 0.599,
                                   'KL145': 0.8723,
                                   'KL18': 0.5384,
                                   'KL43': 0.5604,
                                   'KL5': 0.6075,
                                   'KL51': 0.8183,
                                   'KL57': 0.9685,
                                   'KL64': 0.5089,
                                   'KL7': 0.9996,
                                   'KL70': 0.8128,
                                   'KL74': 0.66,
                                   'KL8': 0.9349,
                                   'KL9': 0.9793},
 'AB716666.1_prot_BAP15746.1_34': {'KL102': 0.6198,
                                   'KL111': 0.5726,
                              

In [8]:
predictions = [other_predictions]

with open("/media/concha-eloko/Linux/PPT_clean/TropiGAT.Others.all.finetuned_lvl_9.2411.results.tsv", "w") as outfile:
    for prediction in predictions:
        prediction_sorted = dict(sorted(prediction.items()))
        for prot in prediction_sorted:
            if prediction_sorted[prot] == "No hits" or len(prediction_sorted[prot]) == 0:
                outfile.write(f"{prot}\tNo hits\n")
            else:
                outfile.write(f"{prot}\t")
                hits = [f"{kltype}:{round(score, 3)}" for kltype, score in prediction_sorted[prot].items()]
                sorted_hits = " ; ".join(sorted(hits, key=lambda x: float(x.split(":")[1]), reverse=True))
                outfile.write(sorted_hits)
                outfile.write("\n")

In [ ]:
other_res_df = pd.read_csv(f"/media/concha-eloko/Linux/PPT_clean/other_res.df.tsv", sep = "\t", header = 0)
df_sorted = other_res_df.sort_values(by='Proteins', ascending=True)

In [ ]:
df_sorted.to_csv(f"/media/concha-eloko/Linux/PPT_clean/other_ordered.df.tsv", sep = "\t", header = 0, index = False)